In [4]:
# pip install pymorphy2

  Using cached pymorphy2-0.8-py2.py3-none-any.whl (46 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
Processing /home/sde/.cache/pip/wheels/9b/04/dd/7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e/docopt-0.6.2-py2.py3-none-any.whl
  Using cached pymorphy2_dicts-2.4.393442.3710985-py2.py3-none-any.whl (7.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import sys
import re
from os import listdir, getcwd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pymorphy2 import MorphAnalyzer

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/sde/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/sde/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
stop_words = stopwords.words('russian')+['это', 'свой']

In [22]:
morph_analyzer = MorphAnalyzer()

In [4]:
def lemmatize(word): #приводим слова в нормальную форму
    parse = morph_analyzer.parse(word)[0]
    return(parse.normal_form+'_'+parse.tag.POS)
        
def tokenize(text): #разбиваем текст на слова
    if type(text) == str:
        words = [i for i in word_tokenize(text) if i not in stop_words and not i.isdigit()]
        words = [lemmatize(i) for i in words if morph_analyzer.parse(i)[0].tag.POS != None]
        return words
    else: return ''

In [5]:
import requests
import json
storage_service_api = 'http://127.0.0.1:5730'
analytics_service_api = 'http://127.0.0.1:5731'


def get_records(work_place, role, date_time_start, date_time_end):
    headers = {
            'Content-Type': 'application/json'
        }
    payload = {
        'work_place': work_place,
        'role': role,
        'date_time_start': date_time_start,
        'date_time_end': date_time_end,
    }
    r = requests.get(analytics_service_api+'/analyse',
                      params=payload,
                      headers=headers).json()
    return r


results = get_records(work_place=1, role=0, date_time_start='2020-02-16', date_time_end='2020-06-20')

[[1, 1, '0', 'Mon, 15 Jun 2020 10:52:13 GMT', '1(234)567-89-10'], [2, 1, '0', 'Mon, 15 Jun 2020 10:52:13 GMT', '1(234)567-89-10'], [3, 1, '0', 'Mon, 15 Jun 2020 11:43:02 GMT', 'А ты 4567-89-10 с'], [4, 1, '0', 'Mon, 15 Jun 2020 12:19:37 GMT', '1234-56-78 910-11-12'], [5, 1, '0', 'Mon, 15 Jun 2020 10:52:13 GMT', '1(234)567-89-10'], [6, 1, '0', 'Mon, 15 Jun 2020 11:43:02 GMT', 'А ты 4567-89-10 с'], [7, 1, '0', 'Mon, 15 Jun 2020 12:19:37 GMT', '1234-56-78 910-11-12'], [15, 1, '0', 'Wed, 17 Jun 2020 00:05:44 GMT', 'Эй что происходит я тебя понял'], [16, 1, '0', 'Wed, 17 Jun 2020 00:05:13 GMT', 'Что то знаешь у**** н**** устроил'], [17, 1, '0', 'Wed, 17 Jun 2020 00:02:28 GMT', 'Крит'], [18, 1, '0', 'Wed, 17 Jun 2020 00:25:15 GMT', 'Сейчас ничего нибудь пизданется полюбому ничего'], [19, 1, '0', 'Wed, 17 Jun 2020 00:02:59 GMT', 'Я тебя понял'], [20, 1, '0', 'Mon, 15 Jun 2020 12:19:37 GMT', '1234-56-78 910-11-12'], [21, 1, '0', 'Wed, 17 Jun 2020 00:05:34 GMT', '1 2 3'], [22, 1, '0', 'Wed, 17 

In [20]:
data_all = pd.DataFrame(results, columns = ['id', 'work_place', 'role', 'date', 'text'])

In [23]:
data_proc = data_all.copy() #создаем рабочий экзмепляр данных

data_proc['text'] = [tokenize(i) for i in data_all['text']] #токенизируем текст, замещаем в датафрейме
data_proc = data_proc[data_proc['text'].apply(len) != 0] #убираем пустые текста

In [25]:
all_words_with_meta = [] #создаем промежуточный список для последующего создания датафрейма
for sample in data_proc.iterrows(): #идем по датафрейму, выбираем слова, забираем их параметры
    for word in sample[1]['text']:
        all_words_with_meta.append([word, sample[1]['work_place'], sample[1]['role'], sample[1]['date']])

In [26]:
all_words_with_meta = np.array(all_words_with_meta) #переводим в массив numpy
df_words = pd.DataFrame({'word':all_words_with_meta[:, 0],
                        'work_place':all_words_with_meta[:, 1],
                       'role':all_words_with_meta[:, 2],
                        'date':all_words_with_meta[:, 3]})

In [27]:
df_words

,word,work_place,role,date
0,а_CONJ,1,0,"Mon, 15 Jun 2020 11:43:02 GMT"
1,а_CONJ,1,0,"Mon, 15 Jun 2020 11:43:02 GMT"
2,эй_INTJ,1,0,"Wed, 17 Jun 2020 00:05:44 GMT"
3,происходить_VERB,1,0,"Wed, 17 Jun 2020 00:05:44 GMT"
4,понять_VERB,1,0,"Wed, 17 Jun 2020 00:05:44 GMT"
5,что_CONJ,1,0,"Wed, 17 Jun 2020 00:05:13 GMT"
6,знаешь_CONJ,1,0,"Wed, 17 Jun 2020 00:05:13 GMT"
7,устроить_VERB,1,0,"Wed, 17 Jun 2020 00:05:13 GMT"
8,крит_NOUN,1,0,"Wed, 17 Jun 2020 00:02:28 GMT"
9,сейчас_ADVB,1,0,"Wed, 17 Jun 2020 00:25:15 GMT"


In [28]:
df_words['word_clean'] = df_words['word'].apply(lambda x: re.sub(r'(_[^_]*$)', '', x)) #чистим слова от частей речи

In [31]:
#предварительно отсортируем рабочие места, роли и даты для правильной конкатенации
work_places = list(set(df_words['work_place']))
work_places.sort()
dates = list(set(df_words['date']))
dates.sort()
roles = list(set(df_words['role']))
roles.sort()

In [32]:
#функция расчета частоты
def get_freq(work_place='all', date='all'):
    word_freqs = []
    if work_place == 'all' and date == 'all':
        words = df_words['word']
    elif date== 'all':
        words = df_words['word'][df_words['work_place'] == work_place]
    elif work_place == 'all':
        words = df_words['word'][df_words['date'] == date]
    else:
        words = df_words['word'][df_words['work_place'] == work_place and df_words['date'] == date]    
    freqs_dist = nltk.FreqDist(words)
    for word in words:
        word_freqs.append(freqs_dist[word])
    return word_freqs, list((np.array(word_freqs)/len(words)))

In [33]:
#общая частота
freq_common = get_freq()
df_words['freqs_common'], df_words['freqs_common_scaled'] = freq_common[0], freq_common[1]

In [35]:
df_words.sort_values('work_place', inplace = True)

#считаем частоты слов по группам и датам
work_place_id = []

In [36]:
i = 0
for work_place in work_places:
    for work_place_ in df_words['work_place']:
        if work_place_ == work_place:
            work_place_id.append(i)
    i += 1
    
df_words['work_place_id'] = work_place_id

In [38]:
freq_work_places = []
freq_work_places_scaled = []

for work_place in work_places:
    freq_work_places_all = get_freq(work_place=work_place)
    freq_work_places += freq_work_places_all[0]
    freq_work_places_scaled += freq_work_places_all[1]

df_words['freq_work_places'] = freq_work_places
df_words['freq_work_places_scaled'] = freq_work_places_scaled

In [40]:
freq_dates = []
freq_dates_scaled = []

df_words.sort_values('date', inplace = True)
for date in dates:
    freq_dates_all = get_freq(date=date)
    freq_dates += freq_dates_all[0]
    freq_dates_scaled += freq_dates_all[1]

df_words['freqs_date'] = freq_dates
df_words['freqs_date_scaled'] = freq_dates_scaled

In [42]:
df_words.to_csv('dataset.csv')